In [1]:
# Импортируем библиотеки.

import pandas as pd
import numpy as np

In [2]:
# Считываем исходный файл в дф

df = pd.read_csv('test_data.csv')

In [3]:
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


### Задача №1. 
#### Извлечь реплики, в которых менеджер поздоровался с клиентом.

In [4]:
# Произведем фильтрацию исходного дф по полю role, оставив только реплики менеджеров.
# Далее будем работать с этим дф.

df_manager = df.query('role == "manager"').reset_index(drop=True)
df_manager.head(5)

,dlg_id,line_n,role,text
0,0,1,manager,Алло здравствуйте
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
2,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
3,0,8,manager,Угу а на что вы обращаете внимание при выборе
4,0,11,manager,Что для вас приоритет


In [5]:
# Напишем функцию get_greeting, которая возвращает True, если в реплике
# менеджера присутствуют приветственные слова и False в противном случае.

def get_greeting(phrase_list):
    
    # Проверяем вхождение ключевых слов в реплику менеджера
    if 'здравствуйте' in phrase_list:
        return True
    elif 'добрый' in phrase_list and 'день' in phrase_list:
        return True
    elif 'добрый' in phrase_list and 'вечер' in phrase_list:
        return True
    elif 'доброе' in phrase_list and 'утро' in phrase_list:
        return True
    elif 'приветствую' in phrase_list:
        return True
    else:
        return False

In [6]:
# Создаем новый столбец was_greeting, применив функцию get_greeting к текстовому столбцу df_manager.

df_manager['was_greeting'] = df_manager.text.str.lower().str.split().apply(get_greeting)

In [7]:
# Наконец, фильтруем дф по созданному столбцу is_greeting и выводим уникальные приветственные реплики

print(*df_manager.query('was_greeting == True').text.unique(), sep='\n')

Алло здравствуйте
Алло дмитрий добрый день


### Задача №2. 
#### Извлечь реплики, в которых менеджер представился клиенту.

In [8]:
# Напишем функцию get_introduced, которая возвращает True, если в реплике
# менеджера присутствуют ключевые слова и False в противном случае.

def get_introduced(phrase_list):
    
    # Проверяем вхождение ключевых слов в реплику менеджера
    if 'меня' in phrase_list and 'зовут' in phrase_list:
        return True
    elif 'мое' in phrase_list and 'имя' in phrase_list:
        return True
    elif 'да' in phrase_list and 'это' in phrase_list:
        if phrase_list.index('да') - phrase_list.index('это') == -1:
            return True
    else:
        return False

In [9]:
# Создаем новый столбец was_introduced, применив функцию get_introduced к текстовому столбцу df_manager.

df_manager['was_introduced'] = df_manager.text.str.lower().str.split().apply(get_introduced)

In [11]:
# Наконец, фильтруем дф по созданному столбцу was_introduced и выводим реплики, в которых менеджер представился клиенту.

print(*df_manager.query('was_introduced == True').text.unique(), sep='\n')

Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается
Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там
Добрый меня максим зовут компания китобизнес удобно говорить
Да это анастасия


### Задача №3. 
#### Необходимо извлечь имя менеджера.

In [12]:
# Напишем функцию get_manager_name, которая возвращает имя менеджера, 
# если в реплике менеджера присутствуют ключевые слова.

def get_manager_name(phrase_list):
    
    # Проверяем вхождение ключевых слов в реплику менеджера
    if 'меня' in phrase_list and 'зовут' in phrase_list:
        # Проверяем последовательность слов в реплике
        if phrase_list.index('меня') - phrase_list.index('зовут') == -1:
            manager_name = phrase_list.pop(phrase_list.index('зовут') + 1)
            return manager_name.capitalize()
        elif phrase_list.index('меня') - phrase_list.index('зовут') == -2:
            manager_name = phrase_list.pop(phrase_list.index('зовут') - 1)
            return manager_name.capitalize()
    elif 'да' in phrase_list and 'это' in phrase_list:
        if phrase_list.index('да') - phrase_list.index('это') == -1:
            manager_name = phrase_list.pop(phrase_list.index('это') + 1)
            return manager_name.capitalize()

In [13]:
# Создаем новый столбец manager_name, применив функцию get_manager_name к текстовому столбцу df_manager.

df_manager['manager_name'] = df_manager.text.str.lower().str.split().apply(get_manager_name)

In [14]:
# Выводим все имена менеджеров, которые представились клиентам.

print(*df_manager[~df_manager.manager_name.isnull()].manager_name.unique(), sep=', ')


Ангелина, Максим, Анастасия


### Задача №4. 
#### Необходимо извлечь название компании.

In [15]:
# Напишем функцию get_company_name, которая возвращает название компании, 
# если оно упоминалось в реплике менеджера.

def get_company_name(phrase_list):
    
    # Проверяем вхождение ключевых слов в реплику менеджера
    if 'компания' in phrase_list:
        company_name = phrase_list.pop(phrase_list.index('компания') + 1)
        if company_name == 'диджитал':
            return 'Диджитал бизнес'
        elif company_name == 'китобизнес':
            return 'Китобизнес'

In [16]:
# Создаем новый столбец company_name, применив функцию get_company_name к текстовому столбцу df_manager.

df_manager['company_name'] = df_manager.text.str.lower().str.split().apply(get_company_name)

In [17]:
# Выводим все названия компаний, которые менеджеры представили во время разговора.

print(*df_manager[~df_manager.company_name.isnull()].company_name.unique(), sep=', ')

Диджитал бизнес, Китобизнес


### Задача №5. 
#### Извлечь реплики, в которых менеджер попрощался с клиентом.

In [18]:
# Напишем функцию get_goodbye, которая возвращает True, если в реплике
# менеджера присутствуют ключевые слова и False в противном случае.

def get_goodbye(phrase_list):
    
    # Проверяем вхождение ключевых слов в реплику менеджера
    if 'до' in phrase_list and 'свидания' in phrase_list:
        return True
    elif 'всего' in phrase_list and 'доброго' in phrase_list:
        return True
    elif 'до' in phrase_list and 'встречи' in phrase_list:
        return True
    else:
        return False

In [19]:
# Создаем новый столбец said_goodbye, применив функцию get_goodbye к текстовому столбцу df_manager.

df_manager['said_goodbye'] = df_manager.text.str.lower().str.split().apply(get_goodbye)

In [20]:
# Наконец, фильтруем дф по созданному столбцу said_goodbye и выводим прощальные реплики.

print(*df_manager.query('said_goodbye == True').text.unique(), sep='\n')

Всего хорошего до свидания
До свидания
Угу все хорошо да понедельника тогда всего доброго
Во вторник все ну с вами да тогда до вторника до свидания
Ну до свидания хорошего вечера


### Задача №6. 
#### Проверка требований к менеджерам.

In [21]:
# Необходимо проверить следующее требование к менеджеру: 
# «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [22]:
df_manager

,dlg_id,line_n,role,text,was_greeting,was_introduced,manager_name,company_name,said_goodbye
0,0,1,manager,Алло здравствуйте,True,False,None,None,False
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,True,Ангелина,Диджитал бизнес,False
2,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,False,False,None,None,False
3,0,8,manager,Угу а на что вы обращаете внимание при выборе,False,False,None,None,False
4,0,11,manager,Что для вас приоритет,False,False,None,None,False
...,...,...,...,...,...,...,...,...,...
196,5,135,manager,Так дмитрий,False,False,None,None,False
197,5,136,manager,Все записала тогда завтра ждите звонка,False,False,None,None,False
198,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,None,None,False
199,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,None,None,False


In [23]:
# Группируем df_manager по столбцу dlg_id

df_grouped = df_manager.groupby('dlg_id').agg({'was_greeting': 'sum',
                                 'was_introduced': 'sum',
                                 'manager_name': 'unique',
                                 'company_name': 'unique',
                                 'said_goodbye': 'sum'})

In [24]:
# Преобразуем значения в столбцах manager_name и company_name

In [25]:
manager_names = []

for values in df_grouped.manager_name:
    if len(values) == 2:
        for value in values:
            if pd.notnull(value):
                manager_names.append(value)
    else:
        manager_names.append(*values)

In [26]:
company_names = []

for values in df_grouped.company_name:
    if len(values) == 2:
        for value in values:
            if pd.notnull(value):
                company_names.append(value)
    else:
        company_names.append(*values)

In [27]:
df_grouped['manager_name'] = manager_names
df_grouped['company_name'] = company_names

In [28]:
# Сгруппированный дф по dlg_id выглядит следующим образом:

df_grouped

,was_greeting,was_introduced,manager_name,company_name,said_goodbye
dlg_id,,,,,
0,1,1,Ангелина,Диджитал бизнес,1
1,1,1,Ангелина,Диджитал бизнес,1
2,1,1,Ангелина,Диджитал бизнес,0
3,1,1,Максим,Китобизнес,1
4,0,0,None,None,1
5,0,1,Анастасия,None,1


In [29]:
# Произведем проверку требований к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

df_grouped.query('was_greeting == 1 and said_goodbye == 1')

,was_greeting,was_introduced,manager_name,company_name,said_goodbye
dlg_id,,,,,
0,1,1,Ангелина,Диджитал бизнес,1
1,1,1,Ангелина,Диджитал бизнес,1
3,1,1,Максим,Китобизнес,1


In [221]:
# Как видно из таблиц выше, менеджер Ангелина из компании "Диджитал бизнес" поздоровалась и попрощалась с клиентами в двух
# диалогах из трёх, в то время как менеджер Максим из компании "Китобизнес" полностью выполнил требование о приветствии и 
# прощании с клиентом в своём единственном диалоге.